In [1]:
from pynq import Overlay, allocate
import numpy as np

In [7]:
overlay = Overlay('cnn.bit')

In [14]:
def load_fc(load_num):
    t = [1] * 128 + [2] * 128 + [3] * 128 + [4] * 128 + [5] * 128 + [6] * 128 + [7] * 128 + [8] * 128 + [9] * 128 + [10] * 128
   
    OFMCH = 10
    IFMCH = 128

    KerDim = 1
    IFMDim = 1
    OFMDim = 1
    
    kernel = np.append([load_num, 0, KerDim, IFMCH, IFMDim, OFMCH, OFMDim], t)
    print(kernel, len(kernel))

    in_fc = allocate(shape=(len(kernel),), dtype=np.int16)
    out_fc = allocate(shape=(len(kernel),), dtype=np.int16)

    for i, v in enumerate(kernel):
        in_fc[i] = v

    overlay.memory.axi_dma_0.sendchannel.transfer(in_fc)
    overlay.memory.axi_dma_0.recvchannel.transfer(out_fc)
    overlay.memory.axi_dma_0.sendchannel.wait()
    overlay.memory.axi_dma_0.recvchannel.wait()

    print((in_fc == out_fc).all())
    in_fc.close()
    out_fc.close()

In [15]:
load_fc(2)

[ 2  0  1 ... 10 10 10] 1287
True


In [19]:
in_buffer = allocate(shape=(7 + 128,), dtype=np.int16)
out_buffer = allocate(shape=(17,), dtype=np.int16)

input_val = np.append([0, 1, 1, 128, 1, 10, 1],  list(range(-32, 96))[::-1] )

for i, v in enumerate(input_val):
    in_buffer[i] = v


overlay.memory.axi_dma_0.sendchannel.transfer(in_buffer)
overlay.memory.axi_dma_0.recvchannel.transfer(out_buffer)
overlay.memory.axi_dma_0.sendchannel.wait()
overlay.memory.axi_dma_0.recvchannel.wait()


print(out_buffer)

in_buffer.close()
out_buffer.close()

[  0   1   1 128   1  10   1  63 126 189 252 315 378 441 504 567 630]
